In [1]:
path = '/mnt/e/03 - Codes/05 - Zidio Intership/12-Zidio-Projects/01 - Projects/02 - Speech Emotion Recognition/data'

In [3]:
!pwd

/mnt/e/03 - Codes/05 - Zidio Intership/12-Zidio-Projects/01 - Projects/02 - Speech Emotion Recognition


In [6]:
! kaggle datasets download -d ejlok1/toronto-emotional-speech-set-tess -p 'data/'
# ! unzip toronto-emotional-speech-set-tess

Dataset URL: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
100%|█████████████████████████████████████████| 428M/428M [07:12<00:00, 821kB/s]
100%|████████████████████████████████████████| 428M/428M [07:12<00:00, 1.04MB/s]


In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
from keras import utils

warnings.filterwarnings('ignore)

In [ ]:
for dirname, _, filenames in os.walk(''):
    for filename in filenames:
        os.paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.'[0])
        os.labels.append(label.lower())
    
    if len(paths) == 2800:
        break

print('Dataset is loaded')

In [ ]:
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels
df.drop('label count', axis = 1)

In [ ]:
sns.countplot(data = df, x = 'label')